## requirements.txt

In [0]:
streamlit==1.38.0
databricks-sdk>=0.26.0
databricks-vectorsearch>=0.2.0

## app.py

In [0]:
%python
import streamlit as st
import pandas as pd

st.set_page_config(layout="wide")

import streamlit as st
from databricks.sdk import WorkspaceClient
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

# --- Initialize Databricks clients ---
w = WorkspaceClient()
vsc = VectorSearchClient(
    workspace_url=https://12345.gcp.databricks.com,
    personal_access_token="XXXXXXX "
)

# --- Function to retrieve chunks and query the LLM ---

def answer_question(query, index_name, model_name="databricks-meta-llama-3-3-70b-instruct"):

    # Step 1: Retrieve top matching chunks

    index = vsc.get_index(
        endpoint_name="databrew_endpoint",                # the vector search endpoint
        index_name="dev.stranger_things.fake_sow_vsi"  # your index
    )

    results = index.similarity_search(
        query_text=query,
        columns=["id", "text"],
        num_results=3

    )

    if not results:
        return "❌ No relevant context found in the index."

    # Step 2: Combine chunks into context
    # context = "\n\n".join([r["text"] for r in results])
    context = "\n\n".join([row[1] for row in results['result']['data_array']])

    # Step 3: Build prompt for the LLM
    prompt = f"""
        You are a helpful assistant that answers questions using project SOW documents.

        Context:
        {context}

        Question: {query}

        Answer clearly and only use the information from the context above.
    """

    messages = [
        ChatMessage(
            role=ChatMessageRole.SYSTEM,
            content="You are a helpful assistant."
        ),

        ChatMessage(
            role=ChatMessageRole.USER,
            content=prompt
        )
    ]

    # Step 4: Call the Databricks LLM endpoint

    response = w.serving_endpoints.query(
        name=model_name,
        messages=messages,
        max_tokens=512,
        temperature=0.2
    )

    # Step 5: Extract the answer
    try:
        # answer = response.output["message"]["content"][0]["text"]
        answer = response.choices[0].message.content
    except Exception:
        answer = str(response)
 
    return answer

# --- Streamlit UI ---
st.title("📘 Databricks SOW Q&A Assistant")
query = st.text_input("Ask a question about your SOW document:")

if st.button("Get Answer"):

    with st.spinner("Retrieving context and generating answer..."):

        answer = answer_question(query, index_name="sow_vector_index")

    st.subheader("Answer:")

    st.write(answer)